In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tslearn.clustering import TimeSeriesKMeans
import math
import copy
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
def read_data(folder_path="Datasets_CSV/Domain1_csv/"):
    matrix_list = []
    y_list = []
    df_Y= pd.DataFrame(columns=['number'])
    count=0
    len_tab=[]
    for subject in range(10):
        for number in range(10):
            for attemp in range(10):
                
                #we read the data from the specific .csv file
                file_path = folder_path + 'Subject{}-{}-{}.csv'.format(subject+1, number, attemp+1) #To change if your files have different name
                data_X = pd.read_csv(file_path)
                
                count+=len(data_X)
                len_tab.append(len(data_X))
                #Add the specific label to the labels' dataframe
                df_Y = pd.concat([df_Y, pd.DataFrame({'number': [number]})], ignore_index=True) 
                

                #Before the the dataframe converted to array to the matrix list we have to preprocess it:
                
                # Extract the x and y columns
                x = data_X['<x>']
                y = data_X['<y>']
                z = data_X['<z>']
                t = data_X['<t>']

                # Standardize the data
                scaler = StandardScaler()  #TODO: possible option to change ro RobustScaler?
                x = scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
                y = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()
                z = scaler.fit_transform(z.values.reshape(-1, 1)).flatten()
                t = scaler.fit_transform(t.values.reshape(-1, 1)).flatten()

                # Apply PCA with just 2 components so we have just two coordinates (which actually represent X,Y)
                """pca = PCA(n_components=2)
                data_X_standarized = pd.DataFrame({'x': x, 'y': y, 'z': z})
                pca.fit(data_X_standarized)
                data_X_transformed = pca.transform(data_X_standarized) #it returns an array."""

                # DECOMENTE TO NOT USE PCA
                data_X_transformed = pd.DataFrame({'x': x, 'y': y, 'z': z}).to_numpy()
                
                
                #add the preprocessed data to the matrix_list (actually the data is a matrix)
                matrix_list.append(data_X_transformed)
    avg=count/1000
    #convert the List of matrix with the data into a np array
    array_matrix = np.array(matrix_list,dtype=object)
    #convert the dataframe of the Labels into a np array 
    labels = np.array(np.ravel(df_Y))
    return array_matrix,labels,avg,len_tab


arr,lab,avg,len_tab=read_data()
matrix=[]
print(arr.shape)
print(sum(len_tab))
for i in range(len(arr)):
    for j in range(len_tab[i]):
        matrix.append(arr[i][j])
matrix=np.array(matrix,dtype=object)
print(matrix.shape)

(1000,)
85095
(85095, 3)


In [10]:
#------------------ 2D plot ------------------------------

# Create the scatter plot
"""plt.scatter(x, y)

# Add labels and a title
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Scatter plot of X and Y')

# Show the plot
plt.show()"""

#--------------------------- 3D plot ---------------------------

"""fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z)

# Add labels and a title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Scatter plot of X, Y, and Z')

# Show the plot
plt.show()"""

# -------------------- Plot 2D with the PCA --------------------------------------

"""plt.scatter(transformed[:,0], transformed[:,1])

# Add labels and a title
plt.xlabel('PCA_1')
plt.ylabel('PCA_2')
plt.title('Scatter plot of X and Y')

# Show the plot
plt.show()"""

"plt.scatter(transformed[:,0], transformed[:,1])\n\n# Add labels and a title\nplt.xlabel('PCA_1')\nplt.ylabel('PCA_2')\nplt.title('Scatter plot of X and Y')\n\n# Show the plot\nplt.show()"

In [11]:
print(matrix.shape)
cluster_count = 10

X = matrix.reshape(-1, len(matrix[0]))
print(X.shape)
km = TimeSeriesKMeans(n_clusters=cluster_count, metric="dtw")
string = km.fit_predict(X)
np.save('my_array.npy', string)


(85095, 3)
(85095, 3)


In [4]:
string=np.load('my_array.npy')
clust_list=[]
i,j,k=0,0,0
while(i<len(string)):
    row=[]
    j=0
    while(j<len_tab[k]):
        row.append(string[i])
        i+=1
        j+=1
    clust_list.append(row)
    k+=1
clust_list=np.array(clust_list,dtype=object)
print(clust_list.shape)


(1000,)


In [5]:
def edit_distance(a1, a2):
    m, n = len(a1), len(a2)
    # Initialize dynamic programming table
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    # Initialize base cases
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    # Fill in the table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if a1[i - 1] == a2[j - 1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(dp[i - 1][j] + 1,   # deletion
                           dp[i][j - 1] + 1,   # insertion
                           dp[i - 1][j - 1] + cost)  # substitution
    # Return minimum edit distance
    return dp[m][n]

In [6]:
accuracy=[]
# TEST ON ONE SUBJECT
for subject in range(10):
    train_set=copy.deepcopy(clust_list)
    test_set=[]
    test_lab=lab[0:100]
    remove_idx=[]
    # WE REMOVE THE TEST SUBJECT FROM TRAINING DATA
    for idx in range(100):
        test_set.append(train_set[idx+subject*100])
        remove_idx.append(idx+subject*100)
    train_set=np.delete(train_set,remove_idx, axis=0)

    # COMPUTE EDIT DISTANCE BETWEEN TEST AND TRAIN
    matrice=np.empty((len(test_set),10,int(len(train_set)/10)))
    for i in range(len(test_set)):
        count=0
        mult=0
        index=0
        for j in range(len(train_set)):
            if index>=10:
                index=0
            if count>=100:
                mult+=1
                count=0
            matrice[i][lab[j]][index+mult*10]=edit_distance(train_set[j],test_set[i])
            count+=1
            index+=1
    
    # Make the prediction 
    pred=[]
    for i in range(len(test_set)):
        avg=[]
        for j in range(10):
            avg.append(np.average(matrice[i][j]))
        pred.append(np.argmin(avg))

    print("accuracy of the subject ",subject, " : ",accuracy_score(test_lab.tolist(),pred))
    accuracy.append(accuracy_score(test_lab.tolist(),pred))
print(np.mean(accuracy))


accuracy of the subject  0  :  0.78
accuracy of the subject  1  :  0.11
accuracy of the subject  2  :  0.91
accuracy of the subject  3  :  0.3
accuracy of the subject  4  :  0.66
accuracy of the subject  5  :  0.34
accuracy of the subject  6  :  0.59
accuracy of the subject  7  :  0.84
accuracy of the subject  8  :  0.69
accuracy of the subject  9  :  0.77
0.5989999999999999


In [108]:
train_set=copy.deepcopy(clust_list)
test_set=[]
test_lab=[]
remove_idx=[]
for i in range(10):
    removed_row = train_set[0+i*10:3+i*10]
    test_set.append(removed_row[0])
    test_set.append(removed_row[1])
    test_set.append(removed_row[2])
    test_lab.append(lab[0+i*10])
    test_lab.append(lab[1+i*10])
    test_lab.append(lab[2+i*10])
    remove_idx.append(0+i*10)
    remove_idx.append(1+i*10)
    remove_idx.append(2+i*10)

train_set = np.delete(train_set,remove_idx, axis=0)
print(len(train_set))
print(len(test_set))


70
30


In [109]:
matrice=np.empty((len(test_set),10,int(len(train_set)/10)))
for i in range(len(test_set)):
    count=0
    testlab=0
    for j in range(len(train_set)):
        if count>=len(train_set)/10:
            count=0
            test_lab+=1
        matrice[i][test_lab][count]=edit_distance(train_set[j],test_set[i])
        count+=1

print(matrice.shape)

(30, 10, 7)


In [110]:
pred=[]
for i in range(len(test_set)):
    avg=[]
    for j in range(10):
        avg.append(np.average(matrice[i][j]))
    pred.append(np.argmin(avg))
print(pred)

[0, 8, 5, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9]


In [111]:
accuracy_score(test_lab, pred)

0.9333333333333333